In [48]:
import os
import scipy.io as sio
import numpy as np
import pandas as pd
import time

from cest_mrf.metrics.dot_product import dot_prod_matching
from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.plot_functions import mask_roi_finder
from my_funcs.plot_functions import mask_check_plot
from my_funcs.plot_functions import compare_txt_method
from my_funcs.plot_functions import t1_t2_pixel_reader
from my_funcs.plot_functions import real_t1_t2
from my_funcs.plot_functions import real_t1_t2_bg
from my_funcs.plot_functions import dict_t1_t2
from my_funcs.plot_functions import dict_fs_ksw
from my_funcs.plot_functions import norm_pixels
from my_funcs.plot_functions import plot_norm_sig

from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.cest_functions import dicom_data_arranger
from my_funcs.cest_functions import z_spec_rearranger
from my_funcs.cest_functions import offset_rearranger

from my_funcs.b0_mapping_functions import wassr_b0_mapping
from my_funcs.b0_mapping_functions import b0_correction


## 1.1. Enter Data: ##

In [49]:
phantom_choice = 2  # choose 1-3


In [50]:
# Root stats:
general_fn = os.path.abspath(os.curdir)
# Subject stats:
glu_phantom_fns = [os.path.join(general_fn, 'scans', 'data_glutamate_phantoms_jan16_2024',
                                '20240116_091014_OrPerlman_phantom_glu_10_20_30mM_pH7_a_1_1'),
                   os.path.join(general_fn, 'scans', 'data_glutamate_phantoms_jan16_2024',
                                '20240116_114710_OrPerlman_phantom_glu_5_10_15mM_pH7_a_1_1'),
                   os.path.join(general_fn, 'scans', 'data_glutamate_phantoms_jan16_2024',
                                '20240116_132701_OrPerlman_phantom_glu_20mM_pH6p8_7_7p2_a_1_1')]  # January
glu_phantom_fn = glu_phantom_fns[phantom_choice-1]
txt_file_name = 'labarchive_notes.txt'
fp_prtcl_name = '107d'  # to be looped later with all options


## 1.2. Create bruker dataset and masks ##

In [51]:
glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, fp_prtcl_name)

_, _, bruker_dataset_mask = bruker_dataset_creator(glu_phantom_fn, txt_file_name, '107a')  # always takes mask from 107a
vial_rois, full_mask, bg_mask = mask_roi_finder(bruker_dataset_mask)

quant_data_fn = os.path.join(glu_phantom_mrf_files_fn, 'quant_maps.mat')
quant_maps = sio.loadmat(quant_data_fn)
fs_array = quant_maps['fs'] * 110e3 / 3
ksw_array = quant_maps['ksw']
dp_array = quant_maps['dp']
t1w_array = quant_maps['t1w'] * 1000
t2w_array = quant_maps['t2w'] * 1000


In [52]:
# # Specify the file path for saving the MATLAB file
# matlab_file_path = os.path.join(general_fn, 'b0_correction', 'masks', f'phantom_{phantom_choice}_mask')
# # Save the NumPy array as a MATLAB file
# sio.savemat(matlab_file_path, {f'phantom_{phantom_choice}_mask': full_mask})
    

## 2.1. plot 1- mask check ##

In [53]:
conc_ls = [['30 mM', '20 mM', '10 mM'], ['10 mM', '5 mM', '15 mM'], ['20 mM', '20 mM', '20 mM']]  # this is the order of the masks using 107a
ph_ls = [['pH 7', 'pH 7', 'pH 7'], ['pH 7', 'pH 7', 'pH 7'], ['pH 7', 'pH 7.2', 'pH 6.8']]  # this is the order of the masks using 107a

tags = [['a', 'b', 'c'], ['b', 'c', 'a'], ['b', 'a', 'c']]

tag_x_locs = [[-3,3,-5], [-2,3,-2], [3, -3, 5]]
tag_y_locs = [[-13,13,12], [-14,13,13], [-13, 13, 13]]

conc_l = conc_ls[phantom_choice-1]
ph_l = ph_ls[phantom_choice-1]
tag = tags[phantom_choice-1]
tag_x_loc = tag_x_locs[phantom_choice-1]
tag_y_loc = tag_y_locs[phantom_choice-1]

mask_check_plot(vial_rois, full_mask, bg_mask, conc_l, ph_l)


## 2.2. plot 2- parameter check ##

In [54]:
prtcl_txt_fn = os.path.join(general_fn, 'protocol_text_files', 'glutamate', f'{fp_prtcl_name}.txt')

compare_txt_method(prtcl_txt_fn, bruker_dataset, fp_prtcl_name)


The protocol '107d' matches between scan data and txt file


## 2.3. t1 & t2 check ##

In [55]:
t1_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_phantom_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t1')
t2_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_phantom_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t2')

real_t1_t2(t1_pixels, t2_pixels, full_mask, vial_rois, tag, tag_x_loc, tag_y_loc, phantom_choice, conc_l, ph_l)
real_t1_t2_bg(t1_pixels, t2_pixels, bg_mask, phantom_choice)
dict_t1_t2(t1w_array, t2w_array, full_mask, vial_rois, tag, tag_x_loc, tag_y_loc, fp_prtcl_name, phantom_choice, conc_l, ph_l)


## 3. fs & ksw maps ##

In [56]:
dict_fs_ksw(fs_array, ksw_array, full_mask, vial_rois, tag, tag_x_loc, tag_y_loc, fp_prtcl_name, phantom_choice, conc_l, ph_l, [[0, 35], [3000, 10000]])


## 4. Signal close look ##

In [57]:
import numpy as np

# # pixel choice (bottom right)
expected_fs = np.array([[30, 20, 10], [10, 5, 15], [20, 20, 20]]) * 3 / 110e3
expected_ks = [[7500, 7500, 7500], [7500, 7500, 7500], [7500, 9000, 5500]]

expected_f = expected_fs[phantom_choice-1]
expected_k = expected_ks[phantom_choice-1]
# expected_k = np.array([3000, 3000, 3000])

# expected_t1 = np.array([3500, 3500, 3500]) / 1000
# expected_t2 = np.array([1600, 1600, 1600]) / 1000
# expected_t1 = np.array([2900, 2900, 2900]) / 1000
# expected_t2 = np.array([1100, 1100, 1100]) / 1000
expected_t1 = np.array([3600, 3450, 3350]) / 1000  # vial t1s (phantom 2)
expected_t2 = np.array([800, 1000, 600]) / 1000  # vial t2s (phantom 2)

expected_t1 = np.array([3200, 3200, 3200]) / 1000  # bg t1s (phantom 2)
expected_t2 = np.array([1700, 1700, 1700]) / 1000  # bg t2s (phantom 2)
t1 = 2800
t2 = 1750
# expected_t1 = np.array([t1, t1, t1]) / 1000
# expected_t2 = np.array([t2, t2, t2]) / 1000

sig_0 = norm_pixels(0, full_mask, vial_rois, quant_data_fn, glu_phantom_mrf_files_fn, expected_f, expected_k, expected_t1, expected_t2)  # real, matched, expected
sig_1 = norm_pixels(1, full_mask, vial_rois, quant_data_fn, glu_phantom_mrf_files_fn, expected_f, expected_k, expected_t1, expected_t2)  # real, matched, expected
sig_2 = norm_pixels(2, full_mask, vial_rois, quant_data_fn, glu_phantom_mrf_files_fn, expected_f, expected_k, expected_t1, expected_t2)  # real, matched, expected

plot_norm_sig(tag, conc_l, ph_l, sig_0, sig_1, sig_2, fp_prtcl_name, phantom_choice)


In [58]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

abc_id = [tag.index('a'), tag.index('b'), tag.index('c')]  # [2, 0, 1]
abc_int = np.argsort(np.argsort(tag))  # [1, 2, 0]

fig = make_subplots(rows=1, cols=1, vertical_spacing=0.07, horizontal_spacing=0.01,
                    subplot_titles=f'')

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
sigs = [sig_0, sig_1, sig_2]

x_axis = np.arange(0, len(sig_0[0]))
for i, id in enumerate(abc_id):  # from large to small
    sig = sigs[id]
    scatter_real = go.Scatter(x=x_axis, y=sig[0], mode='lines', name=f'Real {conc_l[id]}, {ph_l[id]}', line=dict(color=colors[i]))
    scatter_match = go.Scatter(x=x_axis, y=sig[2], mode='lines', name=f'Expected {conc_l[id]}, {ph_l[id]}', line=dict(color=colors[i], dash='dot'))
    # scatter_match = go.Scatter(x=x_axis, y=sig[1], mode='lines', name=f'Matched {conc_l[id]}, {ph_l[id]}', line=dict(color=colors[i], dash='dot'))
    
    fig.add_trace(scatter_real, row=1, col=1)
    fig.add_trace(scatter_match, row=1, col=1)

fig.update_yaxes(row=1, col=1, title_text=f'Normalized signal')

# Set layout for better visualization
fig.update_layout(
    template='plotly_white',  # Set the theme to plotly white
    title_text=f'{fp_prtcl_name}, signal close look',
    showlegend=True,  # Hide legend
    legend=dict(x=1, y=1.05),
    height=400,
    width=700  # Set a width based on your preference
)

labels_to_show_in_legend = ['Real Signal', 'Matched Signal']

# only show 3 first traces in legend!
for trace_i, trace in enumerate(fig['data']):
    if trace_i > 6:
        trace['showlegend'] = False

# Adjust margin to reduce whitespace
fig.update_layout(margin=dict(l=0, r=5, t=60, b=0))
# Set y-axis ticks to values in the range [0, 1] with a step of 0.2
# fig.update_yaxes(tickmode='linear', tick0=0, dtick=0.1, range=[0.05, 0.4])

# Show the plot
fig.show()

pio.write_image(fig, f'images/phantom_{phantom_choice}/{fp_prtcl_name}_sigs.jpeg')
